In [1]:
from pathlib import Path
import numpy as np
import torch
import padertorch as pt
import padercontrib as pc
import paderbox as pb
from padertorch import Model
from sacred import Experiment, commands
from sklearn import metrics
from tqdm import tqdm
from paderbox.io.new_subdir import get_new_subdir
from paderbox.io import load_json, dump_json
from pprint import pprint
from fearless.sid.data import prepare_data
from fearless.sid.data import prepare_data_2
from scipy.special import softmax
from scipy.spatial.distance import euclidean as euc
from statistics import mean
import tensorflow as tf
import pb_sed.evaluation.instance_based as sed
import lazy_dataset
from padertorch.data.utils import collate_fn
from padertorch.ops.sequence.pack_module import pad_sequence
from padercontrib.database.fearless import Fearless
import padertorch as pt
import paderbox as pb
import numpy as np
import paderbox as pb
from padercontrib.database.iterator import AudioReader
from paderbox.transform import stft,fbank
import scipy
import pydub
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
import pandas as pd
from pydub import AudioSegment
import torch
from padertorch.data.utils import collate_fn
import lazy_dataset
from padertorch.ops.sequence.pack_module import pad_sequence
import pandas
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from fearless.sid.data import prepare_data_3
from fearless.sid.data import prepare_data_4

In [3]:
db = Fearless()

In [4]:
def prepare_data(example, batch_size, shuffle = False):
    

    ds = example
    
    def prep_features(example):
        padded_audio = []
        fbank_data = []
        
        """ Obtain audio segments from the dataset"""
        """ If segments smaller than 4secs, pad with silence. Else, extract 4secs from larger audio segments """
        audio_sam = AudioSegment.from_wav(example['audio_path']['observation'])
        if audio_sam.duration_seconds < 4:
            pad_ms = (4 - audio_sam.duration_seconds)*1000
            silence = AudioSegment.silent(duration=pad_ms) # milliseconds of silence needed
            padded = audio_sam + silence
            
        elif audio_sam.duration_seconds >= 4:
            pad_ms = 0
            audio_sam = audio_sam[0:4000]
            silence = AudioSegment.silent(duration=pad_ms)
            padded = audio_sam + silence        
  
        a = padded.get_array_of_samples()
        b = np.array(a)
        padded_audio.append(b)
        
        """ Compute the 64 dimensional filter banks for the 4secs fixed length audio segments"""
    
        f_banks = fbank(padded_audio, sample_rate=8000, window_length=400, stft_shift=160, number_of_filters=64,
                        stft_size=512,lowest_frequency=0,highest_frequency=None, preemphasis_factor=0.97,
                        window=scipy.signal.windows.hamming, denoise=False)
        fbank_data.append(f_banks)
        float_fbank = np.float32(fbank_data)
        float_fbank = np.squeeze(float_fbank,0)
        
        example['features'] = (float_fbank)
    
        return example
    
    def prep_label(example):
        #label_dict = pb.io.load_json(path = '/net/home/dheerajpr/my_project/fearless/fearless/sid/labels_argmax.json')
        label_hot = pb.io.load_json(path = '/net/home/dheerajpr/my_project/fearless/fearless/sid/labels_167_hot.json')
        label_dict3 = pb.io.load_json(path = '/net/home/dheerajpr/my_project/fearless/fearless/sid/labels_167.json')
        if example['speaker_id'] in label_dict3.keys():
        
            pos = label_dict3[example['speaker_id']]
            example['label_array'] = np.array(pos)
            
        if example['speaker_id'] in label_hot.keys():
        
            pos = label_hot[example['speaker_id']]
            example['label_hot'] = np.array(pos)  
    
        return example
        
    
    def stack(example):
        
        example['features'] = np.stack(example['features'])
        example['label_array'] = np.stack(example['label_array'])
        example['label_hot'] = np.stack(example['label_hot'])
        
        return example
    
    example = example.map(prep_features)
    example = example.map(prep_label)
    if shuffle:
        example = example.shuffle()
   
    example = example.batch(batch_size).map(collate_fn)
    example = example.map(stack)
    #example = example.prefetch(buffer_size=8, num_workers=8)
              
            
            
    return example

In [5]:
dev_167 = pb.io.load_json("/net/home/dheerajpr/my_project/fearless/fearless/sid/dev_167.json")
train_167 =  pb.io.load_json("/net/home/dheerajpr/my_project/fearless/fearless/sid/train_167.json")
dev_sid = db.get_dataset("Dev_SID")
train_sid =   db.get_dataset("Train_SID")
#dev_50 = pb.io.load_json("/net/home/dheerajpr/my_project/fearless/fearless/sid/dev_50.json")
train_167_lazy = lazy_dataset.new(train_167['Train_167'])
dev_167_lazy = lazy_dataset.new(dev_167['Dev_167'])
#dev_50_lazy = lazy_dataset.new(dev_50['Dev_50'])
train_data = prepare_data_4(train_167_lazy,batch_size=1)
validation_data = prepare_data_4(dev_167_lazy,batch_size=1)

In [130]:
#exp_dir = '/net/vol/dheerajpr/models/SID/2021-08-03-10-24-31' #167
#exp_dir = '/net/vol/dheerajpr/models/SID/2021-08-19-11-13-00' #167 better 3sec
#exp_dir = '/net/vol/dheerajpr/models/SID/2021-08-12-13-23-49' #Model strides at 2
#exp_dir = '/net/vol/dheerajpr/models/SID/2021-08-12-16-42-54' #LR 0.05, layer 3 and 4 stride 1
#exp_dir = '/net/vol/dheerajpr/models/SID/2021-09-04-14-41-51' #167 new 32k data prep validated on train segments

#exp_dir = '/net/vol/dheerajpr/models/SID/2021-08-12-21-03-14' #Best result yet(1)
#exp_dir = '/net/vol/dheerajpr/models/SID/2021-08-13-10-17-52' #(2)
#exp_dir = '/net/vol/dheerajpr/models/SID/2021-08-10-12-43-44' #New var length



#exp_dir = '/net/vol/dheerajpr/models/SID/2021-09-05-02-45-38' # 32k proper
#exp_dir = '/net/vol/dheerajpr/models/SID/2021-09-05-22-42-44' # Segmented (1)

#exp_dir = '/net/vol/dheerajpr/models/SID/2021-09-08-23-13-35' # Segmented (2)
#exp_dir = '/net/vol/dheerajpr/models/SID/2021-09-11-12-33-27' # Segmented (3)


#exp_dir = '/net/vol/dheerajpr/models/SID/2021-09-28-22-38-53' #100 spk

exp_dir = '/net/vol/dheerajpr/models/SID/2021-09-09-20-08-35' #SID dataset
#exp_dir = '/net/vol/dheerajpr/models/SID/2021-10-02-10-29-26'  #|SID BIG MODEL

#exp_dir = '/net/vol/dheerajpr/models/SID/2021-10-11-18-57-39' # SID shift 160 to 180
#exp_dir = '/net/vol/dheerajpr/models/SID/2021-10-12-23-24-11' # SID logfbanks
#ckpt_name = 'ckpt_48000.pth'

#exp_dir = '/net/vol/dheerajpr/models/SID/2021-10-13-10-35-12' # 167 logfbanks

#exp_dir = '/net/vol/dheerajpr/models/SID/2021-10-14-20-49-27' # SID logfbanks 5secs
#ckpt_name = 'ckpt_51000.pth'
#exp_dir = '/net/vol/dheerajpr/models/SID/2021-10-15-21-17-48'
#ckpt_name = 'ckpt_48280.pth'
ckpt_name = 'ckpt_best_loss.pth'
device = 0
model_SID = Model.from_storage_dir(
    exp_dir, consider_mpi=True, checkpoint_name=ckpt_name
)
model_SID.to(device)
model_SID.eval()

ResNet_SID(
  size=ModelParameterSize(total_count=5388634, trainable_count=5388634, total_bytes=21554536, trainable_bytes=21554536)
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (avgpool2d): AvgPool2d(kernel_size=3, stride=2, padding=1)
  (layer1): Sequential(
    (0): Block(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): LeakyReLU(negative_slope=0.01)
    )
    (1): Block(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(

In [100]:
del exp_dir

In [133]:
def evaluation_data(example,batch_size=1,shuffle=False):
    
    def prep_features(example):
        
        fbank_data = []
        audio = pb.io.load_audio(example['audio_path']['observation'],dtype=np.int16)
        #audio = example['array_samples']
        f_banks = pb.transform.fbank(audio, sample_rate=8000, window_length=400, stft_shift=160, number_of_filters=64,
                            stft_size=512,lowest_frequency=0,highest_frequency=None, preemphasis_factor=0.97,
                            window=scipy.signal.windows.hamming, denoise=False)
        fbank_data.append(f_banks)
        float_fbank = np.float32(fbank_data)
        example['features'] = torch.from_numpy(float_fbank)
        
        return example
    
    def prep_label(example):
        label_dict = pb.io.load_json(path = '/net/home/dheerajpr/my_project/fearless/fearless/sid/labels_sid.json')
        label_dict2 = pb.io.load_json(path = '/net/home/dheerajpr/my_project/fearless/fearless/sid/labels_sid_hot.json')
        
        if example['speaker_id'] in label_dict.keys():
        
            pos = label_dict[example['speaker_id']]
            
            example['label_array'] = np.array(pos) 
            
        if example['speaker_id'] in label_dict2.keys():
        
            pos = label_dict2[example['speaker_id']]
            #pos = np.concatenate((pos,np.zeros(51)))
            example['label_hot'] = np.array(pos)
        return example
        
    
     
    def stack(example):
        example['features'] = pad_sequence(example['features'], batch_first=True)
        #example['features'] =  torch.unsqueeze(example['features'],1)
        example['label_array'] = np.stack(example['label_array'])
        example['label_hot'] = np.stack(example['label_hot'])
        
        return example
    
    example = example.map(prep_features)
    example = example.map(prep_label)    
    
    if shuffle:
        example = example.shuffle()
    example = example.batch(batch_size).map(collate_fn)
    example = example.map(stack)          
            
    return example

In [ ]:
labels_100 = pb.io.load_json("/net/home/dheerajpr/my_project/fearless/fearless/sid/labels_100.json")
labels_50 = pb.io.load_json("/net/home/dheerajpr/my_project/fearless/fearless/sid/labels_50.json")

In [ ]:
df = pd.DataFrame(dev_sid)

In [ ]:
df_100 = df[df['speaker_id'].isin(labels_100.keys())]
list_100 = pd.DataFrame.to_dict(df_100,'records')
df_50 = df[df['speaker_id'].isin(labels_50.keys())]
list_50 = pd.DataFrame.to_dict(df_50,'records')
ds_100 = lazy_dataset.new(list_100)
ds_50 = lazy_dataset.new(list_50)

In [134]:
#eval_dev_sid = evaluation_data(dev_sid)
eval_dev = evaluation_data(dev_sid,batch_size=1)
eval_train =  evaluation_data(train_167_lazy)
eval_dev

            DictDataset(name='Dev_SID', len=6373)
          MapDataset(_pickle.loads)
        MapDataset(<function evaluation_data.<locals>.prep_features at 0x7f38fe5a6680>)
      MapDataset(<function evaluation_data.<locals>.prep_label at 0x7f38fe5a6710>)
    BatchDataset(batch_size=1)
  MapDataset(<function collate_fn at 0x7f3a19a49710>)
MapDataset(<function evaluation_data.<locals>.stack at 0x7f38fe5a6e60>)

In [87]:
eval_dev[0]

{'audio_path': {'observation': ['/net/db/fearless/Audio/Segments/SID/Dev/FS02_SID_dev_0001.wav']},
 'num_samples': [25760],
 'speaker_id': ['NETWORK5'],
 'example_id': ['FS02_SID_dev_0001'],
 'dataset': ['Dev_SID'],
 'features': tensor([[[[ 1.9220,  7.3107,  8.6591,  ...,  6.0031,  6.5285,  5.1012],
           [ 2.3762,  7.5539,  8.8342,  ...,  5.4482,  5.6939,  4.3270],
           [ 2.4039,  7.7061,  9.0035,  ...,  6.5840,  6.2537,  4.1797],
           ...,
           [ 2.3579,  7.9967, 11.2510,  ...,  7.6528,  6.7941,  5.7264],
           [ 4.7742,  6.3954,  9.8499,  ...,  5.2437,  5.6866,  4.3517],
           [ 1.3189,  6.9546,  8.0696,  ...,  5.2239,  5.3703,  3.6732]]]]),
 'label_array': array([142]),
 'label_hot': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

# Evaluation for the train segment dataset

In [135]:
#Evaluate

with torch.no_grad():
        metric = {'Accuracy':[],'Top_5_Accuracy':[]}
        Accuracy = []
        Precision = []
        Recall = []
        F1 = []
        Top_5 = []
        Top_2 = []
        Top_1 = []
        scores = []
        targets = []
        fbank_data = []
        
        for example in tqdm(eval_dev):          
            
            example = model_SID.example_to_device(example, device)
            output = model_SID(example)            
            pred = output['prediction'].cpu().detach().numpy()
            prediction_soft = softmax(pred)
            prediction_max = np.argmax(pred, axis=-1)
            target = example['label_array'].cpu().detach().numpy()
            target_hot = example['label_hot'].cpu().detach().numpy()
            #print(target_hot)
            accuracy = (prediction_max == target).mean()            
           
            Accuracy.append(accuracy)  
            
            top_5 = tf.keras.metrics.sparse_top_k_categorical_accuracy(target,prediction_soft, k=5)
            top_5 = np.array(top_5).mean()
            Top_5.append(top_5)
            
            top_2 = tf.keras.metrics.sparse_top_k_categorical_accuracy(target,prediction_soft, k=2)
            top_2 = np.array(top_2).mean()
            Top_2.append(top_2)
            
            top_1 = tf.keras.metrics.sparse_top_k_categorical_accuracy(target,prediction_soft, k=1)
            top_1 = np.array(top_1).mean()
            Top_1.append(top_1)
            
            precision = metrics.precision_score(target,prediction_max, average='macro', zero_division=0)
            Precision.append(precision)
            recall = metrics.recall_score(target,prediction_max, average='macro', zero_division=0)
            Recall.append(recall)
            f1 = metrics.f1_score(target,prediction_max, average='macro', zero_division=0)
            F1.append(f1)
            
            targets.append(target_hot)            
            scores.append(prediction_soft)
            
        targets_con = np.concatenate((targets))
        scores_con = np.concatenate((scores))
        #thr,met = sed.get_optimal_thresholds(targets_con,scores_con,metric='f1')       
        #decisions = scores_con > thr
        #print(decisions)
        #f1, p, r = sed.fscore(targets, decisions, event_wise=False)
       # auc = metrics.roc_auc_score(targets_con, scores_con, None)
                    
        metric['Accuracy'] = np.mean(Accuracy)
        metric['Top_5_Accuracy'] = np.mean(Top_5)
        metric['Top_2_Accuracy'] = np.mean(Top_2)
        metric['Top_1_Accuracy'] = np.mean(Top_1)
        metric['Precision'] = np.mean(Precision)
        metric['Recall'] = np.mean(Recall)
        metric['F1_score'] = np.mean(F1)
        #metric['AUC'] = np.mean(auc)
        #metric['Thresholds'] = thr
        
        pprint(metric)

100%|██████████| 6373/6373 [02:19<00:00, 45.74it/s]


{'Accuracy': 0.6635807312097913,
 'F1_score': 0.6635807312097913,
 'Precision': 0.6635807312097913,
 'Recall': 0.6635807312097913,
 'Top_1_Accuracy': 0.6635807,
 'Top_2_Accuracy': 0.7693394,
 'Top_5_Accuracy': 0.86552644}


In [59]:
metric

{'Accuracy': 0.768868664679115,
 'Top_5_Accuracy': 0.9165228,
 'Top_2_Accuracy': 0.85218894,
 'Top_1_Accuracy': 0.7688687,
 'Precision': 0.768868664679115,
 'Recall': 0.768868664679115,
 'F1_score': 0.768868664679115}

In [103]:
targets_con.shape, scores_con.shape

((6373, 218), (6373, 218))

In [17]:
decisions = scores_con>thr

NameError: name 'thr' is not defined

In [104]:
f1, p, r = sed.fscore(targets_con, decisions,beta=1.0, event_wise=False)
f1,p,r

NameError: name 'decisions' is not defined

In [136]:
targets_argmax = []
scores_argmax = []
for i in targets_con:
    targets_argmax.append(np.argmax(i))

for i in scores_con:
    scores_argmax.append(np.argmax(i))

In [137]:
print(metrics.f1_score(targets_argmax,scores_argmax,average='weighted'), ':', 'weighted')
print(metrics.f1_score(targets_argmax,scores_argmax,average='macro',zero_division=0), ':', 'macro')
print(metrics.f1_score(targets_argmax,scores_argmax,average='micro'), ':', 'micro')

0.652318935993544 : weighted
0.3467637752558192 : macro
0.6635807312097913 : micro


In [107]:
cnf_matrix = metrics.confusion_matrix(targets_argmax,scores_argmax)

In [21]:
FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
TP = np.diag(cnf_matrix)
TN = cnf_matrix.sum() - (FP + FN + TP)
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)
# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)
# Overall accuracy for each class
ACC = (TP+TN)/(TP+FP+FN+TN)
#DCF for each class
DCF = (0.75*FNR)+(0.25*FPR)

/net/software/python/2020_06/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  
/net/software/python/2020_06/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


In [22]:
min(DCF), np.argmin(DCF)

(0.00011775788977861517, 1)

In [112]:
num_samples = []
for i in dev_sid:
    num_samples.append(i['num_samples'])

num_samples = [x / 8000 for x in num_samples]

In [113]:
new_dict = {}
new_dict['duration'] = num_samples
new_dict['targets'] = targets_argmax
new_dict['scores'] = scores_argmax

pd = pandas.DataFrame.from_dict(new_dict)
pd

,duration,targets,scores
0,3.22,142,196
1,4.18,19,19
2,3.47,70,70
3,4.22,73,73
4,4.47,73,167
...,...,...,...
6368,3.31,201,201
6369,4.23,19,19
6370,4.14,19,19
6371,4.45,19,19


In [114]:
comparison_column = np.where(pd['targets']==pd['scores'],True, False)
pd["match"] = comparison_column

In [115]:
duration = pd[pd["match"]==True]

pd_sort_dur = pd.sort_values(by=['duration'])


In [116]:
def dur_acc(dur):
    dur_res =  pd_sort_dur.loc[(pd["duration"]>=dur)]        
    return len(dur_res[dur_res["match"]==True])/len(dur_res)

In [117]:
y = []
for i in np.arange(0,20,0.5):
    y.append(dur_acc(i))
    
x = np.arange(0,20,0.5)


ZeroDivisionError: division by zero

In [118]:
plt.plot(x,y)
plt.xlabel("Duration (s)"),plt.ylabel('Accuracy'),plt.title('Duration vs Accuracies for Train Segments')
plt.show()

NameError: name 'x' is not defined

In [119]:
# Ground-truth speaker and count

unique, counts = np.unique(targets_argmax, return_counts=True)
dict1 = dict(zip(unique, counts))


In [ ]:
v = [i for i, j in zip(new_dict['targets'],new_dict['scores']) if i == j]  #Correctly classified samples based on speakers

In [ ]:
# Correctly classified speaker and count

unique_1, counts_1 = np.unique(v, return_counts=True)
dict2 = dict(zip(unique_1, counts_1))

In [ ]:
for key, value in dict1.items():
    print(key, ' : ', value)  #Ground truth speakers and their segment count

In [ ]:
for key, value in dict2.items():
    print(key, ' : ', value)   #Correctly classified speakers and their segment count

In [ ]:
#Accuracies for each class/speaker.
import operator

accs = []
for i in dict1.keys():
    if i in dict2.keys():
        accs.append(dict2[i]/dict1[i])
    
d = dict(zip(unique_1,accs))
#cd = sorted(d.items(),key=operator.itemgetter(1),reverse=True)
d

In [ ]:
# Mean of the accuracies
mean(accs)

# Evaluation for the validation segment dataset

In [ ]:
from fearless.sid.segmenter import segmenter

In [ ]:
dev_167_lazy[8997]

In [ ]:
def chunk_data(example,segment_size=32000):
    
    to_chunk = example
    segmented = segmenter(to_chunk,segment_size)
    segmented = lazy_dataset.new(segmented)
    eval_segmented = evaluation_data(segmented)
    
    return eval_segmented

In [ ]:
#eval_data = chunk_data(dev_167_lazy[8997:8998],segment_size=int(4000*3.21)) #buzz_longest in dev segment ds (3.21)
eval_data = chunk_data(dev_167_lazy[8:9],segment_size=int(4000*8.385))

In [ ]:
eval1 = []
for i in np.arange(0.5,20,0.5):
    eval1.append(chunk_data(dev_167_lazy[8:9],segment_size=int(4000*i)))

In [ ]:
#Evaluate

with torch.no_grad():
        metric = {'Accuracy':[],'Top_5_Accuracy':[]}
        Accuracy = []
        Precision = []
        Recall = []
        F1 = []
        Top_5 = []
        scores = []
        targets = []
        fbank_data = []
        
        for example in tqdm(eval_data):
                
                #example = eval_dev[8]
                example = model_SID.example_to_device(example, device)
                output = model_SID(example)            
                pred = output['prediction'].cpu().detach().numpy()
                prediction_soft = softmax(pred)
                prediction_max = np.argmax(pred, axis=-1)
                target = example['label_array'].cpu().detach().numpy()
                target_hot = example['label_hot'].cpu().detach().numpy()
                #print(np.argmax(prediction_soft))
                accuracy = (prediction_max == target).mean()            
                #print(accuracy)
                Accuracy.append(accuracy)  
                top_5 = tf.keras.metrics.sparse_top_k_categorical_accuracy(target,prediction_soft, k=5)
                top_5 = np.array(top_5).mean()
                Top_5.append(top_5)
                
                precision = metrics.precision_score(target,prediction_max, average='weighted', zero_division=0)
                Precision.append(precision)
                recall = metrics.recall_score(target,prediction_max, average='weighted', zero_division=0)
                Recall.append(recall)
                f1 = metrics.f1_score(target,prediction_max, average='weighted', zero_division=0)
                F1.append(f1)
                
                targets.append(target_hot)            
                scores.append(prediction_soft)
                
        targets_con = np.concatenate((targets))
        scores_con = np.concatenate((scores))
        #thr,met = sed.get_optimal_thresholds(targets_con,scores_con,metric='f1')       
        #decisions = scores_con > thr
        #print(decisions)
        #f1, p, r = sed.fscore(targets, decisions, event_wise=False)
        #auc = metrics.roc_auc_score(targets_con, scores_con, None)
                    
        metric['Accuracy'] = np.mean(Accuracy)
        metric['Top_5_Accuracy'] = np.mean(Top_5)
        metric['Precision'] = np.mean(Precision)
        metric['Recall'] = np.mean(Recall)
        metric['F1_score'] = np.mean(F1)
        #metric['AUC'] = np.mean(auc)
        #metric['Thresholds'] = thr
        
        pprint(metric)

In [ ]:
mean(Accuracy)

In [ ]:
targets_con.shape, scores_con.shape

In [ ]:
decisions = scores_con>thr

In [ ]:
f1, p, r = sed.fscore(targets_con, decisions,beta=1.0, event_wise=False)
f1,p,r

In [ ]:
targets_argmax = []
scores_argmax = []
for i in targets_con:
    targets_argmax.append(np.argmax(i))

for i in scores_con:
    scores_argmax.append(np.argmax(i))

In [ ]:
print(metrics.f1_score(targets_argmax,scores_argmax,average='weighted'), ':', 'weighted')
print(metrics.f1_score(targets_argmax,scores_argmax,average='macro'), ':', 'macro')
print(metrics.f1_score(targets_argmax,scores_argmax,average='micro'), ':', 'micro')

In [ ]:
cnf_matrix = metrics.confusion_matrix(targets_argmax,scores_argmax)

In [ ]:
FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
TP = np.diag(cnf_matrix)
TN = cnf_matrix.sum() - (FP + FN + TP)
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)
# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)
# Overall accuracy for each class
ACC = (TP+TN)/(TP+FP+FN+TN)
#DCF for each class
DCF = (0.75*FNR)+(0.25*FPR)

In [ ]:
num_samples = []
for i in dev_167_lazy:
    num_samples.append(i['num_samples'])

num_samples = [x / 8000 for x in num_samples]

In [ ]:
dev_df = pandas.DataFrame(dev_167_lazy)

In [ ]:
new_dict = {}
new_dict['audio_file'] = dev_df['example_id']
new_dict['duration'] = num_samples
new_dict['targets'] = targets_argmax
new_dict['scores'] = scores_argmax
new_dict
df = pandas.DataFrame.from_dict(new_dict)
df1 = df[df['targets']==df['scores']]

buzz_df = df1[df1['targets']==12]

buzz_dict = pandas.DataFrame.from_dict(buzz_df)

buzz_dict

In [ ]:
comparison_column = np.where(pd['targets']==pd['scores'],True, False)
pd["match"] = comparison_column

duration = pd[pd["match"]==True]

pd_sort_dur = pd.sort_values(by=['duration'])


In [ ]:
def dur_acc(dur):
    dur_res =  pd_sort_dur.loc[(pd["duration"]>=dur)]        
    return len(dur_res[dur_res["match"]==True])/len(dur_res)

In [ ]:
y = []
for i in np.arange(0.5,20,0.5):
    y.append(dur_acc(i))

x = np.arange(0.5,20,0.5)

In [ ]:
plt.plot(x,y)
plt.xlabel("Duration (s)"),plt.ylabel('Accuracy'),plt.title('Duration vs Accuracies for Dev Segments')
plt.show()

In [ ]:
# Ground-truth speaker and count

unique, counts = np.unique(targets_argmax, return_counts=True)
dict1 = dict(zip(unique, counts))


In [ ]:
v = [i for i, j in zip(new_dict['targets'],new_dict['scores']) if i == j]  #Correctly classified samples based on speakers

In [ ]:
# Correctly classified speaker and count

unique_1, counts_1 = np.unique(v, return_counts=True)
dict2 = dict(zip(unique_1, counts_1))

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
metrics.balanced_accuracy_score(targets_argmax,scores_argmax)

In [ ]:
for key, value in dict1.items():
    print(key, ' : ', value)  #Ground truth speakers and their segment count

In [ ]:
for key, value in dict2.items():
    print(key, ' : ', value)   #Correctly classified speakers and their segment count

In [ ]:
#Accuracies for each class/speaker.
import operator

accs = []
for i in dict1.keys():
    if i in dict2.keys():
        accs.append(dict2[i]/dict1[i])
    
d = dict(zip(unique_1,accs))
#cd = sorted(d.items(),key=operator.itemgetter(1),reverse=True)
d

In [ ]:
# Mean of the accuracies
mean(accs)